# Interactive Functions 

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display


from shared_utils import geography_utils, styleguide
from shared_utils import calitp_color_palette as cp 


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We rec

In [2]:
import _dla_utils

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [4]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [5]:
df.sample(5)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10378,Obligated,STPL,6430(001),Alameda County Transportation Improvement Auth...,2014-12-10,2014-12-10,2014-12-10,2014-12-10,2014-12-11,-516555.49,0.00,0.00,Authorized,6430,4,E-76 approved on E-76 approved on,0.00,0.00,1.00,nan,I-580-castro Valley Interchange,Reconfigure On And Off Ramps,2,NaT,NaT,MTC,6430,2014.00,Alameda County Transportation Improvement Auth...
1865,Obligated,STPCML,6084(167),Metropolitan Transportation Commission,2017-10-18,2017-10-18,2018-04-10,2018-04-18,2018-04-19,-0.01,0.00,0.00,Authorized,6084,4,E-76 approved on,NaN,182.00,1.00,REG090045,San Francisco Bay Area,Translink Fare Collection System Fy 2010/2011 ...,3,NaT,NaT,MTC,6084,2017.00,Metropolitan Transportation Commission
14732,Obligated,ATPL,6300(003),University Of California - Santa Barbara,2019-09-18,2019-09-18,2019-09-18,2019-09-18,2019-09-19,0.00,0.00,0.00,Authorized,6300,5,E-76 approved on,6.00,0.00,1.00,UCSB03,Ucsb Campus Trail Connect Isla Vista Elementar...,Construct Multi-modal Trail (tc),4,2019-09-12,2019-09-13,SBCG,6300,2019.00,University of California - Santa Barbara
14618,Obligated,ACSTP,32L0(506),Butte County,2019-09-09,2019-09-09,2019-09-19,2019-09-19,2019-09-23,0.00,37053.00,37053.00,Authorized,5912,3,E-76 approved on,5.00,10.00,4.00,nan,Dillard Rd Sacramento County,Reconstruct The Road Shoulder Using Rip Rap,1,2019-09-04,2019-09-06,BCAG,32L0,2019.00,Butte County
6356,Obligated,BRLS,5094(067),Vacaville,2016-07-08,2016-07-11,2016-07-12,2016-07-12,2016-07-12,3262153.00,0.00,3885210.00,Authorized,5094,4,E-76 approved on Prepared on 10/6/2015,3.00,1.00,0.00,SOL110005,Jepson Parkway: Leisure Town Road Bridge At Ne...,Bridge Replacement,1,NaT,NaT,MTC,5094,2016.00,Vacaville


## Full Function for Prefix

In [7]:
    @interact
    
    def dla_get_prefix(place=df.primary_agency_name.sort_values().unique().tolist()):
        
        agencies = df[df.primary_agency_name==place]
    
        prefix_count_n = agencies >> count(_.prefix)
    
        display(Markdown(f"**Summary Statistics for {place}**"))
        display(Markdown(f"The number of obligations {place} has is {len(agencies)}"))
        
        display(Markdown(f"The number of prefix codes {place} uses is {len(prefix_count_n)}"))
        
        
        
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        display(df[df.agency == place][['fed_requested','ac_requested','total_requested']].describe())
        
        display(Markdown(f"**Top Project Types in {place}**"))
        display((df[df.primary_agency_name == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
         

    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        
        ax1 = (styleguide.preset_plotnine_config(ax1)
         + scale_fill_manual(values=cp.CALITP_CATEGORY_BOLD_COLORS)
        )
        
        return ax1

        

interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Alameda', 'Al…

## Function for results by Prefix

In [8]:
@interact
def prefix_all_agencies_4(prefix_unique=df.prefix.sort_values().unique().tolist()):
    
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n))
    
    prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix_unique', options=('6140000', 'ACNHPI', 'ACSTER', 'ACSTP', '…

## Function for Type of Work

In [9]:
top50 = (df >> count(_.type_of_work) >> arrange(-_.n)).head(50)

In [11]:
@interact
def findtypeofwork(work=top50.type_of_work.sort_values().unique().tolist()):
    
    display(Markdown(f"**Number of Obligations the top Agencies have in {work}**"))
    # graphs 
    worktypes = df[df.type_of_work== work]
    
    work_count = (worktypes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    display(work_count)

    ax1 = (work_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Agencies using Type of Work', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1

interactive(children=(Dropdown(description='work', options=('Ac Overlay', 'Ac Overlay (tc)', 'Asphalt Concrete…

## Function to Filter Data

In [12]:
df_query = df[['prefix', 'primary_agency_name', 'project_no','locode','dist','fed_requested', 'ac_requested', 'total_requested',
              'type_of_work','project_location', 'seq', 'mpo','prepared_y']]

In [13]:
#adding year
@interact
def get_query2(agency=df_query.primary_agency_name.unique().tolist(),
             prefix=df_query.prefix.unique().tolist(),
             year=df_query.prepared_y.unique().tolist()):
   
    filtering = df_query[(df_query.primary_agency_name==agency)&(df_query.prefix==prefix)&(df_query.prepared_y==year)]
    
    
    display(Markdown(f"**The number of obligations {agency} has in {prefix} during the year {year} is {len(filtering)}**"))
    
    display(filtering.head())
    
    

interactive(children=(Dropdown(description='agency', options=('Humboldt County', 'Mendocino County', 'Sacramen…